In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 13:04:30.831902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 13:04:30.935697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 13:04:30.935713: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 13:04:30.957274: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 13:04:31.487687: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Charge',
       'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 13:04:36.051687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 13:04:36.052234: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 13:04:36.052285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 13:04:36.052325: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 13:04:36.052364: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 393ms/step - loss: 2.0315 - auc: 0.4149
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4407 - auc: 0.7738
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.4624 - auc: 0.8404
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5395 - auc: 0.8559
Epoch 5/100
1/1 [==============================] - 0s 25ms/step - loss: 1.5443 - auc: 0.8666
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4654 - auc: 0.8714
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3295 - auc: 0.8826
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1594 - auc: 0.8996
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0723 - auc: 0.8991
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.0008 - auc: 0.9039
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9855 

1/1 [==============================] - 0s 27ms/step - loss: 0.2467 - auc: 0.9896
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2453 - auc: 0.9896
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2417 - auc: 0.9909
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2411 - auc: 0.9903
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2334 - auc: 0.9928
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2639 - auc: 0.9861
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2478 - auc: 0.9893
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2352 - auc: 0.9912
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2456 - auc: 0.9898
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2361 - auc: 0.9913
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Charge',
      'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 341ms/step - loss: 1.5105 - auc: 0.7911
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3183 - auc: 0.7669
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3038 - auc: 0.7158
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1707 - auc: 0.7932
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1172 - auc: 0.8334
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0573 - auc: 0.8641
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0053 - auc: 0.8789
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9472 - auc: 0.8906
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 0.8717 - auc: 0.9064
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 0.8373 - auc: 0.9089
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2714 - auc: 0.9880
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2808 - auc: 0.9853
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2845 - auc: 0.9847
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2778 - auc: 0.9860
Epoch 92/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2686 - auc: 0.9881
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2647 - auc: 0.9888
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2667 - auc: 0.9885
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2660 - auc: 0.9889
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2728 - auc: 0.9868
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2697 - auc: 0.9879
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [5]:
m='remove_Charge'
a=['AtomPairFP',
       'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 4.1419 - auc: 0.2060
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 2.4645 - auc: 0.2924
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4516 - auc: 0.6628
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2186 - auc: 0.8440
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2987 - auc: 0.8725
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4538 - auc: 0.8819
Epoch 7/100
1/1 [==============================] - 0s 44ms/step - loss: 1.5565 - auc: 0.8792
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6111 - auc: 0.8775
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6247 - auc: 0.8761
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6104 - auc: 0.8791
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2618 - auc: 0.9910
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2685 - auc: 0.9900
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2654 - auc: 0.9899
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2484 - auc: 0.9929
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2673 - auc: 0.9891
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2545 - auc: 0.9919
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2606 - auc: 0.9901
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2583 - auc: 0.9905
Epoch 96/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2576 - auc: 0.9906
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2542 - auc: 0.9910
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [6]:
m='remove_MAP4'
a=['AtomPairFP',
       'Charge','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 1.4434 - auc: 0.7513
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3896 - auc: 0.6970
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2306 - auc: 0.8182
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2117 - auc: 0.8363
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1213 - auc: 0.8534
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.0283 - auc: 0.8652
Epoch 7/100
1/1 [==============================] - 0s 38ms/step - loss: 0.9843 - auc: 0.8650
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9488 - auc: 0.8656
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 0.8802 - auc: 0.8937
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 0.8236 - auc: 0.9152
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 0.2864 - auc: 0.9830
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2768 - auc: 0.9859
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2834 - auc: 0.9842
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2828 - auc: 0.9838
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2861 - auc: 0.9840
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2874 - auc: 0.9827
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2774 - auc: 0.9856
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2795 - auc: 0.9855
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2762 - auc: 0.9863
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2824 - auc: 0.9841
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [7]:
m='remove_MOE'
a=['AtomPairFP',
       'Charge','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 1.4685 - auc: 0.7653
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3534 - auc: 0.7186
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2774 - auc: 0.7816
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1785 - auc: 0.8336
Epoch 5/100
1/1 [==============================] - 0s 41ms/step - loss: 1.1033 - auc: 0.8568
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0265 - auc: 0.8690
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9444 - auc: 0.8894
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9023 - auc: 0.8960
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 0.8592 - auc: 0.9091
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 0.7876 - auc: 0.9340
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2484 - auc: 0.9905
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2417 - auc: 0.9914
Epoch 90/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2464 - auc: 0.9910
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2454 - auc: 0.9908
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2516 - auc: 0.9890
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2426 - auc: 0.9913
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2447 - auc: 0.9906
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2419 - auc: 0.9910
Epoch 96/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2484 - auc: 0.9899
Epoch 97/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2470 - auc: 0.9905
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [8]:
m='remove_MorganFP'
a=['AtomPairFP',
       'Charge','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 1.4499 - auc: 0.7246
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.5060 - auc: 0.8005
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3165 - auc: 0.8154
Epoch 4/100
1/1 [==============================] - 0s 42ms/step - loss: 1.1910 - auc: 0.8071
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1708 - auc: 0.7817
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0868 - auc: 0.8193
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 0.9950 - auc: 0.8673
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 0.9086 - auc: 0.9036
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 0.8889 - auc: 0.9127
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 0.8206 - auc: 0.9319
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2624 - auc: 0.9893
Epoch 89/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2627 - auc: 0.9886
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2542 - auc: 0.9904
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2569 - auc: 0.9898
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2606 - auc: 0.9888
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2612 - auc: 0.9893
Epoch 94/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2604 - auc: 0.9892
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2570 - auc: 0.9900
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2649 - auc: 0.9883
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2637 - auc: 0.9885
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [9]:
m='remove_Topology'
a=['AtomPairFP',
       'Charge','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 1.7859 - auc: 0.7774
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.4713 - auc: 0.7663
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3362 - auc: 0.7282
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3076 - auc: 0.7001
Epoch 5/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1872 - auc: 0.7668
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.0740 - auc: 0.8355
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 1.0027 - auc: 0.8738
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 0.9533 - auc: 0.8959
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9173 - auc: 0.9073
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 0.8593 - auc: 0.9184
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - l

1/1 [==============================] - 0s 23ms/step - loss: 0.2300 - auc: 0.9924
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2333 - auc: 0.9912
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2331 - auc: 0.9916
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2337 - auc: 0.9917
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2316 - auc: 0.9923
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2309 - auc: 0.9926
Epoch 94/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2268 - auc: 0.9931
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2329 - auc: 0.9911
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2355 - auc: 0.9909
Epoch 97/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2386 - auc: 0.9906
Epoch 98/100
1/1 [==============================] - 0s 32ms/step - loss: 

In [10]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('6feature_remove_one2.csv',index=False)
print(de)

               model       auc
0          all_model  0.951188
1  remove_AtomPairFP  0.929344
2      remove_Charge  0.948998
3        remove_MAP4  0.947671
4         remove_MOE  0.950987
5    remove_MorganFP  0.950052
6    remove_Topology  0.951916
